# Лабораторная работа №2
> Выполнили:  
> – 339080, Дьячихин Данила Никитич  
> – 368606, Овчинников Павел Алексеевич

**Цель работы:** построить модели линейной классификации и регрессии.

**Инструменты:** модели классификации (`SGDClassifier`) и регрессии (`SGDRegressor`) из пакета scikit-learn.

Будем использовать датасеты, полученные в результате выполнения лабораторной работы №1: `Advertising_preprocessed.csv` и `heart_preprocessed.csv`

Импортируем необходимые библиотеки и загрузим датасеты

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

advertising = pd.read_csv('Advertising_preprocessed.csv')
heart = pd.read_csv('heart_preprocessed.csv')
print(advertising.info())
print(heart.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         198 non-null    float64
 1   radio      198 non-null    float64
 2   newspaper  198 non-null    float64
 3   sales      198 non-null    float64
 4   n/(n+r)    198 non-null    float64
dtypes: float64(5)
memory usage: 7.9 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       296 non-null    float64
 1   sex       296 non-null    int64  
 2   cp        296 non-null    int64  
 3   trestbps  296 non-null    float64
 4   chol      296 non-null    float64
 5   fbs       296 non-null    int64  
 6   restecg   296 non-null    int64  
 7   thalach   296 non-null    float64
 8   exang     296 non-null    int64  
 9   oldpeak   296 non-null    float6

#   Классификация - SGDClassifier   

Используем `heart_preprocessed.csv`

Основные задачи пункта:  
>1.В качестве loss-функции рассмотреть: `perceptron`, `hinge` и `squared_hinge`.  
>2.Рассмотреть различные варианты регуляризации (penalty): `L1`, `L2` или `Elastic Net`.  
>3.Используя GridSearchCV, определить лучший классификатор.  

Задача модели - прогнозирование наличия или отсутсвия болезни сердца, на основе данных

Разобьем данные на обучающую и тестовую выборки (соотношение - 80/20)

In [ ]:
X_class = heart.drop(columns=['target'])
y_class = heart['target']

X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

Зададим параметры для перебора в классификаторе

1.`loss` (Функция потерь)  
Определяет, как модель оценивает ошибку (разницу между предсказанием и истинным значением).  
Влияет на то, какие свойства данных учитываются при обновлении весов.
> `perceptron`: Использует функцию потерь, аналогичную Перцептрону (корректирует веса только для ошибочных предсказаний).  
> `hinge`: Функция потерь для метода опорных векторов (SVM). Максимизирует разделение между классами.  
> `squared_hinge`: Квадрат "hinge loss", более чувствителен к ошибкам, чем обычный 'hinge'.
  
2.`penalty` (Регуляризация)  
Регуляризация ограничивает значения весов модели, чтобы избежать переобучения.
> `l1`: зануляет некоторые веса (создает разреженные модели).  
> `l2`: стремится уменьшить веса, но не зануляет их.  
> `elasticnet`: Сочетание L1 и L2.  

3.`alpha` (Коэффициент регуляризации)  
Это сила регуляризации. Чем больше значение alpha, тем сильнее модель "штрафует" за большие веса.

In [ ]:
classifier_params = {
    'loss': ['perceptron', 'hinge', 'squared_hinge'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [10000]
}

Применим алгоритм классификации

In [ ]:
classifier = SGDClassifier(random_state=42)
clf_grid = GridSearchCV(classifier, param_grid=classifier_params, cv=5, scoring='f1', return_train_score=True)
clf_grid.fit(X_class_train, y_class_train)

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


GridSearchCV(cv=5, estimator=SGDClassifier(random_state=42),
             param_grid={'alpha': [0.0001, 0.001, 0.01],
                         'loss': ['perceptron', 'hinge', 'squared_hinge'],
                         'max_iter': [10000],
                         'penalty': ['l1', 'l2', 'elasticnet']},
             return_train_score=True, scoring='f1')

Используя GridSearchCV, определим лучший классификатор

In [ ]:
best_classifier = clf_grid.best_estimator_
y_class_pred = best_classifier.predict(X_class_test)
classification_accuracy = accuracy_score(y_class_test, y_class_pred)

print("Лучший классификатор:", best_classifier)
print(f"Точность классификации: {classification_accuracy:.2f}")
print("Параметры лучшего классификатора:", clf_grid.best_params_)

Лучший классификатор: SGDClassifier(alpha=0.01, max_iter=10000, random_state=42)
Точность классификации: 0.90
Параметры лучшего классификатора: {'alpha': 0.01, 'loss': 'hinge', 'max_iter': 10000, 'penalty': 'l2'}


Выведем результаты классификации

In [ ]:
results = []

for params, mean_train_score, mean_test_score in zip(
        clf_grid.cv_results_['params'],
        clf_grid.cv_results_['mean_train_score'],
        clf_grid.cv_results_['mean_test_score']
):
    model = SGDClassifier(**params, random_state=42)
    model.fit(X_class_train, y_class_train)

    y_pred = model.predict(X_class_test)
    test_accuracy = accuracy_score(y_class_test, y_pred)

    results.append({
        'params': params,
        'train_accuracy': mean_train_score,
        'cv_accuracy': mean_test_score,
        'test_accuracy': test_accuracy
    })

sorted_results = sorted(results, key=lambda x: x['test_accuracy'], reverse=True)

for i, res in enumerate(sorted_results, 1):
    print(f"{i}. Test Accuracy: {res['test_accuracy']:.4f}, CV Accuracy: {res['cv_accuracy']:.4f}, Params: {res['params']}")

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


1. Test Accuracy: 0.9167, CV Accuracy: 0.8124, Params: {'alpha': 0.01, 'loss': 'perceptron', 'max_iter': 10000, 'penalty': 'l1'}
2. Test Accuracy: 0.9167, CV Accuracy: 0.8340, Params: {'alpha': 0.01, 'loss': 'hinge', 'max_iter': 10000, 'penalty': 'elasticnet'}
3. Test Accuracy: 0.9000, CV Accuracy: 0.7906, Params: {'alpha': 0.0001, 'loss': 'hinge', 'max_iter': 10000, 'penalty': 'l2'}
4. Test Accuracy: 0.9000, CV Accuracy: 0.8311, Params: {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 10000, 'penalty': 'l1'}
5. Test Accuracy: 0.9000, CV Accuracy: 0.8098, Params: {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 10000, 'penalty': 'l2'}
6. Test Accuracy: 0.9000, CV Accuracy: 0.7463, Params: {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 10000, 'penalty': 'elasticnet'}
7. Test Accuracy: 0.9000, CV Accuracy: 0.8701, Params: {'alpha': 0.01, 'loss': 'hinge', 'max_iter': 10000, 'penalty': 'l2'}
8. Test Accuracy: 0.8833, CV Accuracy: 0.8515, Params: {'alpha': 0.0001, 'loss': 'squared_hinge', 'max_ite

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Выводы по пункту "Классификация":

Как можно заметить, наибольший `Test Accuracy` = 0.9167 имеет модель с параметрами {'alpha': 0.01, 'loss': 'perceptron', 'max_iter': 10000, 'penalty': 'l1'}.  
Но расхождение с `CV Accuracy` = 0.7710 достаточно большое. Это может говорить о переобучении.  
Иная ситуация с моделью, полученной пунктом ранее  
`7. Test Accuracy: 0.9000, CV Accuracy: 0.8473, Params: {'alpha': 0.01, 'loss': 'hinge', 'max_iter': 10000, 'penalty': 'l2'}`  
Её показатели более близки друг к другу, к тому же довольно высокие.  
 А, значит, лучшей она оказалась вполне закономерно.

#   Регрессия - SGDRegressor

Используем `Advertising_preprocessed.csv`

Основные задачи пункта:  
>1.В качестве loss-функции рассмотреть: `squared_error`, `huber` и `epsilon_insensitive`.  
>2.Рассмотреть различные варианты регуляризации (penalty): `L1`, `L2` или `Elastic Net`.  
>3.Используя GridSearchCV, определить лучший регрессор.  

Разобьем данные на обучающую и тестовую выборки (соотношение - 80/20)

In [ ]:
X_reg = advertising.drop(columns=['sales'])
y_reg = advertising['sales']

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

Зададим параметры для перебора в регрессоре

loss:  
>`squared_error` (по умолчанию):Это обычная среднеквадратичная ошибка (MSE).  
>`huber`: Комбинация MSE и MAE. Huber менее чувствителен к выбросам, чем MSE.  
>`epsilon_insensitive`: Игнорирует ошибки, меньшие порога.

In [ ]:
regressor_params = {
    'loss': ['squared_error', 'huber', 'epsilon_insensitive'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [10000]
}

Применим алгоритм регрессора

In [ ]:
regressor = SGDRegressor(random_state=42)
reg_grid = GridSearchCV(regressor, param_grid=regressor_params, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
reg_grid.fit(X_reg_train, y_reg_train)

GridSearchCV(cv=5, estimator=SGDRegressor(random_state=42),
             param_grid={'alpha': [0.0001, 0.001, 0.01],
                         'loss': ['squared_error', 'huber',
                                  'epsilon_insensitive'],
                         'max_iter': [10000],
                         'penalty': ['l1', 'l2', 'elasticnet']},
             return_train_score=True, scoring='neg_mean_squared_error')

Используя GridSearchCV, определим лучший регрессор

In [ ]:
best_regressor = reg_grid.best_estimator_
y_reg_pred = best_regressor.predict(X_reg_test)
regression_mse = mean_squared_error(y_reg_test, y_reg_pred)

print("Лучший регрессор:", best_regressor)
print(f"Среднеквадратичная ошибка регрессии: {regression_mse:.2f}")
print("Параметры лучшего регрессора:", reg_grid.best_params_)

Лучший регрессор: SGDRegressor(alpha=0.01, max_iter=10000, penalty='l1', random_state=42)
Среднеквадратичная ошибка регрессии: 0.12
Параметры лучшего регрессора: {'alpha': 0.01, 'loss': 'squared_error', 'max_iter': 10000, 'penalty': 'l1'}


Выведем результаты регрессии

In [ ]:
results = []

for params, mean_train_score, mean_cv_score in zip(
        reg_grid.cv_results_['params'],
        reg_grid.cv_results_['mean_train_score'],
        reg_grid.cv_results_['mean_test_score']
):
    model = SGDRegressor(**params, random_state=42)
    model.fit(X_reg_train, y_reg_train)

    y_pred = model.predict(X_reg_test)
    test_mse = mean_squared_error(y_reg_test, y_pred)
    test_r2 = r2_score(y_reg_test, y_pred)

    results.append({
        'params': params,
        'train_mse': -mean_train_score,
        'cv_mse': -mean_cv_score,
        'test_mse': test_mse,
        'test_r2': test_r2
    })

sorted_results = sorted(results, key=lambda x: x['test_r2'], reverse=True)

print("Результаты всех моделей:")
for i, res in enumerate(sorted_results, 1):
    print(f"{i}. Test MSE: {res['test_mse']:.4f}, Test R2: {res['test_r2']:.4f}, CV MSE: {res['cv_mse']:.4f}, Params: {res['params']}")

Результаты всех моделей:
1. Test MSE: 0.1162, Test R2: 0.8839, CV MSE: 0.1232, Params: {'alpha': 0.001, 'loss': 'epsilon_insensitive', 'max_iter': 10000, 'penalty': 'l1'}
2. Test MSE: 0.1164, Test R2: 0.8838, CV MSE: 0.1230, Params: {'alpha': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 10000, 'penalty': 'l1'}
3. Test MSE: 0.1170, Test R2: 0.8831, CV MSE: 0.1234, Params: {'alpha': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 10000, 'penalty': 'elasticnet'}
4. Test MSE: 0.1179, Test R2: 0.8823, CV MSE: 0.1243, Params: {'alpha': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 10000, 'penalty': 'l2'}
5. Test MSE: 0.1188, Test R2: 0.8814, CV MSE: 0.1211, Params: {'alpha': 0.01, 'loss': 'squared_error', 'max_iter': 10000, 'penalty': 'l1'}
6. Test MSE: 0.1193, Test R2: 0.8809, CV MSE: 0.1214, Params: {'alpha': 0.001, 'loss': 'squared_error', 'max_iter': 10000, 'penalty': 'l1'}
7. Test MSE: 0.1193, Test R2: 0.8808, CV MSE: 0.1214, Params: {'alpha': 0.0001, 'loss': 'squared_error', 'm

Выводы по пункту "Регрессия":

Как можно заметить, полученная в предыдущем пункте модель  
`5. Test MSE: 0.1188, Test R2: 0.8814, CV MSE: 0.1211, Params: {'alpha': 0.01, 'loss': 'squared_error', 'max_iter': 10000, 'penalty': 'l1'}`  
оказалась лучшей на тренировочных данных.

На тесте лучшие результаты у модели  
`1. Test MSE: 0.1162, Test R2: 0.8839, CV MSE: 0.1232, Params: {'alpha': 0.001, 'loss': 'epsilon_insensitive', 'max_iter': 10000, 'penalty': 'l1'}`